# Manipulate user-defined calendars
This notebook explains how to manage user defined calendars.

## Start by importing the required libraries

In [1]:
from lseg_analytics.reference_data import calendars
from lseg_analytics.reference_data.calendars import Calendar, RestDays, HolidayRule, FullDayDuration, AbsolutePositionWhen, Observance, RelativeRescheduleDescription, CalendarDefinition
from lseg_analytics_basic_client.models import Description
from lseg_analytics.common import WeekDay, ValidityPeriod, Month

## Create a new calendar
Create a new calendar and give it a name, for example 'my_cal'. Update the description if you wish to add additional information.

In [2]:
my_cal = Calendar(
    definition=CalendarDefinition(),
    description=Description(summary="My personal calendar"))
  

## Define the rest days
Select the days of the week to be set as rest days and optionally set the validity period for the rest days. If not set, the validiy period is assumed to be perpetual. 

In [3]:
my_cal.definition.rest_days = [
    RestDays(
        rest_days=[WeekDay.SATURDAY, WeekDay.SUNDAY],
        validity_period=ValidityPeriod(
            start_date="2024-01-01",
            end_date="2024-12-31",
        ),
    )
]

## Define the holiday rules
Add multiple holiday rules as you need. For each rule provide a unique name, the duration (which can be either FullDayDuration or HalfDayDuration) and the validity period. The property 'when' should also be set to describe the type of holiday defined as either AbsolutePositionWhen (for fixed holidays), RelativePositionWhen (for holidays that fall on a particular day of the week) or RelativeToRulePositionWhen (for holidays that are set by reference to another date).

In [4]:
my_cal.definition.holiday_rules = [
    HolidayRule(
        name="New Year's Day",
        duration=FullDayDuration(full_day=1),
        validity_period=ValidityPeriod(
            start_date="2024-01-01",
            end_date="2024-12-31",
        ),
        when=AbsolutePositionWhen(day_of_month=1, month=Month.JANUARY),
    ),
]

## Define the first day of the week for the calendar

In [5]:
my_cal.definition.first_day_of_week = WeekDay.MONDAY

## Save, search for and load calendars
Once you have defined the calendar, you need to save it to your 'HOME' space so it can be accessed again. 

To see all the calendars you have saved, call the search method and specify which space. 'HOME' will search for all the calendars you defined and 'LSEG' will search for all the calendars defined by LSEG. 

To fetch a specific calendar, load it by specifying its name and space.


In [6]:
# Save the calendar
my_cal.save(name="My_Calendar", space="HOME")
 
# Search for calendars of the user's space
calendars.search(spaces=["HOME"])
 
# Load the calendar
my_cal = None
my_cal = calendars.load(name="My_Calendar", space="HOME")


## Add further holiday rules to a calendar
After saving the calendar, you can add further holiday rules by appending them to the list. Make sure to save the calendar again afterwards. 

In [7]:
# Add a holiday rule
my_cal.definition.holiday_rules.append(
    HolidayRule(
        name="My birthday",
        duration=FullDayDuration(full_day=1),
        validity_period=ValidityPeriod(
            start_date="2024-01-01",
            end_date="2024-12-31",
        ),
        when=AbsolutePositionWhen(day_of_month=18, month=Month.JUNE),
    )
)

# Save the updated calendar
my_cal.save()

True

## Display the holiday rules

In [8]:
for rule in my_cal.definition.holiday_rules:
    print(rule.name)

New Year's Day
My birthday


## Use the user-defined calendar in calendar functions
You can use the user-defined calendars in all calendar functions, taken into account the userdefined-rules that have been defined. Below is an example with generate_holidays function

In [9]:
# Generate the list of holidays for 2024
holidays = calendars.generate_holidays(
    calendars=[
        "HOME/My_Calendar"
    ],
    start_date="2024-01-01",
    end_date="2024-12-31"
)
  
# Display the generated holidays
for holiday in holidays:
    print(holiday.date, '-', holiday.names[0].name)

2024-01-01 - New Year's Day
2024-06-18 - My birthday


## If you want to clean up your space, delete the calendar

In [10]:
calendars.delete(name="My_Calendar", space="HOME")

True